<a href="https://colab.research.google.com/github/aman-welzin/rag/blob/main/Hybrid_GRAPH_RAG_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental langchain-neo4j langchain-ollama langchain-huggingface ollama gradio pymupdf transformers chromadb neo4j docling tiktoken yfiles_jupyter_graphs wikipedia json-repair


import os
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

!nvidia-smi

!nohup ollama serve &

!ollama ps

!ollama pull mistral


from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from typing import Tuple, List, Optional

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
import os
from langchain_community.graphs import Neo4jGraph
from langchain_community.graphs import Neo4jGraph
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass
from langchain_community.vectorstores import Neo4jVector
import ollama
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
import re
from typing import List

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import Ollama
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from transformers import pipeline
from google.colab import drive
from langchain_huggingface import HuggingFaceEmbeddings
from docling.document_converter import DocumentConverter
from langchain.schema import Document
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain.graphs import Neo4jGraph
from langchain_ollama import OllamaLLM
import pandas as pd
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain.embeddings import HuggingFaceEmbeddings
from neo4j_graphrag.retrievers import HybridRetriever
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
import ast  # Add this line to import the 'ast' module
import json
from neo4j import GraphDatabase
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

NEO4J_URI="neo4j+s://e3cb3331.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RzNdvZMnFteyovdi_A75W5Hj5S8gkd9Hnl26MxmtFdM"

AUTH=(NEO4J_USERNAME, NEO4J_PASSWORD)
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()

os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

graph = Neo4jGraph()

driver = GraphDatabase.driver(
    uri=os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.5 MB/s eta 0:00:00


In [ ]:

def read_files():
    print("Reading all the files in the directory...")

    csv_path = '/content/cc_feats.csv'

    data = pd.read_csv(csv_path)
    print(f"Loaded {len(data)} rows from the CSV file.")
    return data

raw_document = read_files()
len(raw_document)

if isinstance(raw_document, pd.DataFrame):
    text_list = []
    for _, row in raw_document.iterrows():
        row_data = []
        for col in raw_document.columns:
            value = row[col]
            if pd.notna(value):
                value = str(value)
                row_data.append(f"{col.replace('_', ' ').capitalize()}: {value}")
        text = " | ".join(row_data)
        text_list.append(text)
else:
    text_list = [raw_document]

documents = [Document(page_content=text) for text in text_list]
print("printinf documnets",documents)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
# text_chunks = text_splitter.split_documents(documents)
# print(f"Number of chunks: {len(text_chunks)}")



llm = Ollama(model="mistral")
llm_transformer = LLMGraphTransformer(llm=llm)


graph_documents = llm_transformer.convert_to_graph_documents(documents )

graph_documents

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_index_similarity = Neo4jVector.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name="axis_credit_card_index",
    node_label="Document",
    text_node_property="page_content",  # property name in text_chunks
    embedding_node_property="embedding",
    graph=graph
)

default_cypher = "MATCH (n) OPTIONAL MATCH (n)-[r]->(m) RETURN n, r, m"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

showGraph()


vector_index = Neo4jVector.from_existing_graph(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["page_content"],  # Changed from 'text' to 'page_content'
    embedding_node_property="embedding",
    graph=graph
)

def hybrid_graph_rag_qa(query: str, driver):
    embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    with driver.session() as session:
        session.run("""
    CREATE FULLTEXT INDEX creditCardFulltext IF NOT EXISTS
    FOR (n:Document)
    ON EACH [n.page_content]
""")
    # Step 2: Setup Hybrid Retriever
    retriever = HybridRetriever(
        driver=driver,
        vector_index_name="axis_credit_card_index",
        fulltext_index_name="creditCardFulltext",
        embedder=embedder,
        return_properties=["page_content"],
    )

    # Step 3: Fetch relevant documents
    retriever_result = retriever.search(query_text=query, top_k=5)
    print("Retriever Result:", retriever_result)

    if not retriever_result:
        print("No relevant documents found.")
        return "No relevant documents found."

    # Step 4: Prepare the combined context
  # Example for when retriever_result is a tuple
    # combined_context = ""
    # for doc in retriever_result:
    #     try:
    #         # Replace single quotes with double quotes to ensure valid JSON-like structure
    #         content_str = doc.content.replace("'", "\"")

    #         # Now use ast.literal_eval safely
    #         content_dict = ast.literal_eval(content_str)
    #         combined_context += content_dict.get('page_content', '') + "\n\n"
    #     except Exception as e:
    #         print(f"Error parsing content: {e}")
    #         continue
    # # combined_context = "\n\n".join(ast.literal_eval(doc.content)['page_content'] for doc in retriever_result)
    # Step 5: Setup the prompt
    prompt = PromptTemplate.from_template("""
You are an intelligent assistant answering questions about credit cards using the **official, pre-extracted, and verified information provided below**.

✅ Use **only the context** to answer.
❌ Do not say you lack internet access.
❌ Do not refer the user to a financial advisor or a website.
✅ Do not make assumptions — answer factually, even if partially.

---

📚 Context:
{context}

❓ Question:
{question}

🧠 Answer:
""")

    llm = Ollama(model="mistral")
    chain = LLMChain(llm=llm, prompt=prompt)

    # Step 7: Run the LLM
    response = chain.run({
        "context": retriever_result,
        "question": query
    })

    # Step 8: Print the answer
    print("🔎 Final Answer:\n", response)

    # print("\n🧾 Sources:")
    # for i, doc in enumerate(retriever_result):
    #     # Access the correct element of the tuple (doc[0] is the actual content)
    #     print(f"\nSource {i + 1}:\n{doc[0]}")

    return response
hybrid_graph_rag_qa("What are the top 3 cards for shopping?", driver)


